## SQLAlchemy - using an ORM

### what is an ORM?

ORM stands for Object Relational Mapper

https://en.wikipedia.org/wiki/Object-relational_mapping

```
The heart of the problem involves translating the logical representation 
of the objects into an atomized form that is capable of being stored 
in the database, while preserving the properties of the objects 
and their relationships so that they can be reloaded as objects when needed.
```

### what is SQLAlchemy?

* SQLAlchemy is magic!!  

* Allows you to make database style calls in Python as opposed to SQL.

* Can connect to lots of different databases.  Underlying connection engine for many other Python packages like pandas.

* Can use raw SQL if you want

* Allows you to have an abstraction layer between Python and database

https://www.sqlalchemy.org/

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html#pandas.read_sql

### install SQLAlchemy

```pip install SQLAlchemy```

In [21]:
# Check this out!
# !is a Jupyter magic that calls out to the 
!pip install SQLAlchemy

### make sure it works

In [42]:
import sqlalchemy
sqlalchemy.__version__

'1.1.9'

### create a simple database in memory

In [43]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

In [44]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

### define class representing a simple table

In [45]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

In [46]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

### create the database!!

In [47]:
Base.metadata.create_all(engine)

2017-08-08 19:11:47,644 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-08 19:11:47,646 INFO sqlalchemy.engine.base.Engine ()
2017-08-08 19:11:47,648 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-08 19:11:47,648 INFO sqlalchemy.engine.base.Engine ()
2017-08-08 19:11:47,650 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-08-08 19:11:47,650 INFO sqlalchemy.engine.base.Engine ()
2017-08-08 19:11:47,654 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-08-08 19:11:47,658 INFO sqlalchemy.engine.base.Engine ()
2017-08-08 19:11:47,661 INFO sqlalchemy.engine.base.Engine COMMIT


### create our first User

In [48]:
ed_user = User(name='ed', fullname='Mr. Ed', password='wilbur')

### __repr__ should return a printable representation of the object

In [49]:
ed_user.password

'wilbur'

In [50]:
ed_user.fullname

'Mr. Ed'

### need to start a Session in order to "talk to the database"

so far we have only created an object in Python memory using the class we defined.

In [51]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session() # instance of Session()

### first insert 

In [52]:
session.add(ed_user)
session.commit()

2017-08-08 19:14:52,260 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-08 19:14:52,265 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-08 19:14:52,266 INFO sqlalchemy.engine.base.Engine ('ed', 'Mr. Ed', 'wilbur')
2017-08-08 19:14:52,267 INFO sqlalchemy.engine.base.Engine COMMIT


### query to see if it is there

In [53]:
our_user = session.query(User)
our_user.first()

2017-08-08 19:15:14,747 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-08 19:15:14,750 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
 LIMIT ? OFFSET ?
2017-08-08 19:15:14,751 INFO sqlalchemy.engine.base.Engine (1, 0)


<User(name='ed', fullname='Mr. Ed', password='wilbur')>

### query with a filter

In [54]:
our_user = session.query(User).filter_by(name='ed')
our_user.count()

2017-08-08 19:17:24,569 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2017-08-08 19:17:24,571 INFO sqlalchemy.engine.base.Engine ('ed',)


1

### add a bunch more users


In [55]:
gabe = User(name='Gabe', fullname='Gabriel Fine', password='bart')
charlie = User(name='Charlie', fullname='Charlotte Fine', password='lisa')
session.add_all([ gabe, charlie ] )
session.commit()

2017-08-08 19:17:54,528 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-08 19:17:54,529 INFO sqlalchemy.engine.base.Engine ('Gabe', 'Gabriel Fine', 'bart')
2017-08-08 19:17:54,531 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-08-08 19:17:54,532 INFO sqlalchemy.engine.base.Engine ('Charlie', 'Charlotte Fine', 'lisa')
2017-08-08 19:17:54,533 INFO sqlalchemy.engine.base.Engine COMMIT


### check how many Users are in table

In [56]:
query = session.query(User)
query.count()

2017-08-08 19:18:11,277 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-08 19:18:11,278 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users) AS anon_1
2017-08-08 19:18:11,281 INFO sqlalchemy.engine.base.Engine ()


3

In [57]:
our_user = session.query(User).filter_by(name='ed')
our_user.count()

2017-08-08 19:18:40,040 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?) AS anon_1
2017-08-08 19:18:40,042 INFO sqlalchemy.engine.base.Engine ('ed',)


1

### try raw SQL

In [61]:
from sqlalchemy import text

sql = text('select count(*) from users')
result = session.execute(sql)
result.first()[0] * 10

2017-08-08 19:19:53,818 INFO sqlalchemy.engine.base.Engine select count(*) from users
2017-08-08 19:19:53,820 INFO sqlalchemy.engine.base.Engine ()


30

In [62]:
sql = text('select * from users')
result = session.execute(sql)
for rr in result:
    print(rr)


2017-08-08 19:20:23,533 INFO sqlalchemy.engine.base.Engine select * from users
2017-08-08 19:20:23,534 INFO sqlalchemy.engine.base.Engine ()
(1, 'ed', 'Mr. Ed', 'wilbur')
(2, 'Gabe', 'Gabriel Fine', 'bart')
(3, 'Charlie', 'Charlotte Fine', 'lisa')
